In [1]:
import os


In [13]:
cwd = os.getcwd()

In [2]:
print(cwd)

C:\Users\MaxTak\Desktop\pypro\ahkh


In [5]:
from pytube import YouTube
#yt = YouTube("https://www.youtube.com/watch?v=n06H7OcPd-g")
yt = YouTube("https://www.youtube.com/watch?v=HxI2TxhhS9A")
#yt = yt.get('mp4', '480p')
#yt.download(cwd+'/dwon')
#link = "https://www.youtube.com/watch?v=mpjREfvZiDs"
#https://www.youtube.com/watch?v=HxI2TxhhS9A
#link = "www.youtube.com/watch?v=HxI2TxhhS9A"
#YouTube(link).streams.first().download(cwd+'\down')
#yt.streams.all()
#YouTube(link).streams.all()
yt.title

LiveStreamError: Video is currently being streamed live

In [7]:
# using pafy
import pafy
#url = "https://www.youtube.com/watch?v=bMt47wvK6u0"
url = "https://www.youtube.com/watch?v=HxI2TxhhS9A"
video = pafy.new(url)
video.title , video.viewcount, video.author, video.length
#video.duration, video.likes, video.dislikes
#audiostreams = video.audiostreams
#for a in audiostreams:
#        print(a.bitrate, a.extension, a.get_filesize())


('BBC Arabic Live - البث المباشر لتلفزيون بي بي سي عربي',
 16211589,
 'BBC News عربي',
 0.0)

In [23]:
pafy.backend

'youtube-dl'

In [8]:
pafy.new("https://www.youtube.com/watch?v=HxI2TxhhS9A")

Title: BBC Arabic Live - البث المباشر لتلفزيون بي بي سي عربي
Author: BBC News عربي
ID: HxI2TxhhS9A
Duration: 00:00:00
Rating: None
Views: 16211589
Thumbnail: http://i.ytimg.com/vi/HxI2TxhhS9A/default.jpg

In [26]:
import requests  
import os  
import time  
import json  
import sys  
import subprocess  
import datetime  
import getopt  

In [2]:
import pyaudio
import wave
import os

defaultframes = 512

class textcolors:
    if not os.name == 'nt':
        blue = '\033[94m'
        green = '\033[92m'
        warning = '\033[93m'
        fail = '\033[91m'
        end = '\033[0m'
    else:
        blue = ''
        green = ''
        warning = ''
        fail = ''
        end = ''

recorded_frames = []
device_info = {}
useloopback = False
recordtime = 5

#Use module
p = pyaudio.PyAudio()

#Set default to first in list or ask Windows
try:
    default_device_index = p.get_default_input_device_info()
except IOError:
    default_device_index = -1

#Select Device
print (textcolors.blue + "Available devices:\n" + textcolors.end)
for i in range(0, p.get_device_count()):
    info = p.get_device_info_by_index(i)
    print (textcolors.green + str(info["index"]) + textcolors.end + ": \t %s \n \t %s \n" % (info["name"], p.get_host_api_info_by_index(info["hostApi"])["name"]))

    if default_device_index == -1:
        default_device_index = info["index"]

#Handle no devices available
if default_device_index == -1:
    print (textcolors.fail + "No device available. Quitting." + textcolors.end)
    exit()


#Get input or default
device_id = int(input("Choose device [" + textcolors.blue + str(default_device_index) + textcolors.end + "]: ") or default_device_index)
print ("")

#Get device info
try:
    device_info = p.get_device_info_by_index(device_id)
except IOError:
    device_info = p.get_device_info_by_index(default_device_index)
    print (textcolors.warning + "Selection not available, using default." + textcolors.end)

#Choose between loopback or standard mode
is_input = device_info["maxInputChannels"] > 0
is_wasapi = (p.get_host_api_info_by_index(device_info["hostApi"])["name"]).find("WASAPI") != -1
if is_input:
    print (textcolors.blue + "Selection is input using standard mode.\n" + textcolors.end)
else:
    if is_wasapi:
        useloopback = True;
        print (textcolors.green + "Selection is output. Using loopback mode.\n" + textcolors.end)
    else:
        print (textcolors.fail + "Selection is input and does not support loopback mode. Quitting.\n" + textcolors.end)
        exit()

recordtime = int(input("Record time in seconds [" + textcolors.blue + str(recordtime) + textcolors.end + "]: ") or recordtime)

#Open stream
channelcount = device_info["maxInputChannels"] if (device_info["maxOutputChannels"] < device_info["maxInputChannels"]) else device_info["maxOutputChannels"]
stream = p.open(format = pyaudio.paInt16,
                channels = channelcount,
                rate = int(device_info["defaultSampleRate"]),
                input = True,
                frames_per_buffer = defaultframes,
                input_device_index = device_info["index"],
                as_loopback = useloopback)

#Start Recording
print (textcolors.blue + "Starting..." + textcolors.end)

for i in range(0, int(int(device_info["defaultSampleRate"]) / defaultframes * recordtime)):
    recorded_frames.append(stream.read(defaultframes))
    print (".")

print (textcolors.blue + "End." + textcolors.end)
#Stop Recording

stream.stop_stream()
stream.close()

#Close module
p.terminate()

filename = input("Save as [" + textcolors.blue + "out.wav" + textcolors.end + "]: ") or "out.wav"

waveFile = wave.open(filename, 'wb')
waveFile.setnchannels(channelcount)
waveFile.setsampwidth(p.get_sample_size(pyaudio.paInt16))
waveFile.setframerate(int(device_info["defaultSampleRate"]))
waveFile.writeframes(b''.join(recorded_frames))
waveFile.close()

Available devices:

0: 	 Microsoft Sound Mapper - Input 
 	 MME 

1: 	 Jack Mic (IDT High Definition A 
 	 MME 

2: 	 Microphone Array (IDT High Defi 
 	 MME 

3: 	 Dock Mic (IDT High Definition A 
 	 MME 

4: 	 Microsoft Sound Mapper - Output 
 	 MME 

5: 	 Speakers / Headphones (IDT High 
 	 MME 

6: 	 Communications Headphones (IDT  
 	 MME 

7: 	 Speakers / Headphones (IDT High Definition Audio CODEC) 
 	 Windows WASAPI 

8: 	 Communications Headphones (IDT High Definition Audio CODEC) 
 	 Windows WASAPI 

9: 	 Microphone Array (IDT High Definition Audio CODEC) 
 	 Windows WASAPI 

10: 	 Jack Mic (IDT High Definition Audio CODEC) 
 	 Windows WASAPI 

11: 	 Dock Mic (IDT High Definition Audio CODEC) 
 	 Windows WASAPI 

12: 	 Headphones (HpOut) 
 	 Windows WDM-KS 

13: 	 Microphone Array (MicIn2) 
 	 Windows WDM-KS 

14: 	 Jack Mic (MuxedIn) 
 	 Windows WDM-KS 

15: 	 Dock Mic (MuxedIn) 
 	 Windows WDM-KS 

16: 	 Rec. Playback (MuxedIn) 
 	 Windows WDM-KS 

17: 	 Speakers (Speaker/H

In [4]:
from __future__ import print_function
import json
from os.path import join, dirname
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
import threading

# If service instance provides API key authentication
service = SpeechToTextV1(
    ## url is optional, and defaults to the URL below. Use the correct URL for your region.
    #https://gateway-lon.watsonplatform.net/speech-to-text/api
    #url='https://stream.watsonplatform.net/speech-to-text/api',
    url='https://gateway-lon.watsonplatform.net/speech-to-text/api',
    iam_apikey='wweii6-a76AImdqjPFyPpDQBcCxmq-XRuwI4wkgXVNM6')

# service = SpeechToTextV1(
#     username='YOUR SERVICE USERNAME',
#     password='YOUR SERVICE PASSWORD',
#     url='https://stream.watsonplatform.net/speech-to-text/api')






{
  "models": [
    {
      "name": "en-US_BroadbandModel",
      "language": "en-US",
      "url": "https://gateway-lon.watsonplatform.net/speech-to-text/api/v1/models/en-US_BroadbandModel",
      "rate": 16000,
      "supported_features": {
        "custom_language_model": true,
        "speaker_labels": true
      },
      "description": "US English broadband model."
    },
    {
      "name": "es-ES_NarrowbandModel",
      "language": "es-ES",
      "url": "https://gateway-lon.watsonplatform.net/speech-to-text/api/v1/models/es-ES_NarrowbandModel",
      "rate": 8000,
      "supported_features": {
        "custom_language_model": true,
        "speaker_labels": true
      },
      "description": "Spanish narrowband model."
    },
    {
      "name": "en-GB_BroadbandModel",
      "language": "en-GB",
      "url": "https://gateway-lon.watsonplatform.net/speech-to-text/api/v1/models/en-GB_BroadbandModel",
      "rate": 16000,
      "supported_features": {
        "custom_language_model

NameError: name '__file__' is not defined

In [5]:
models = service.list_models().get_result()
print(json.dumps(models, indent=2))

{
  "models": [
    {
      "name": "en-US_BroadbandModel",
      "language": "en-US",
      "url": "https://gateway-lon.watsonplatform.net/speech-to-text/api/v1/models/en-US_BroadbandModel",
      "rate": 16000,
      "supported_features": {
        "custom_language_model": true,
        "speaker_labels": true
      },
      "description": "US English broadband model."
    },
    {
      "name": "es-ES_NarrowbandModel",
      "language": "es-ES",
      "url": "https://gateway-lon.watsonplatform.net/speech-to-text/api/v1/models/es-ES_NarrowbandModel",
      "rate": 8000,
      "supported_features": {
        "custom_language_model": true,
        "speaker_labels": true
      },
      "description": "Spanish narrowband model."
    },
    {
      "name": "en-GB_BroadbandModel",
      "language": "en-GB",
      "url": "https://gateway-lon.watsonplatform.net/speech-to-text/api/v1/models/en-GB_BroadbandModel",
      "rate": 16000,
      "supported_features": {
        "custom_language_model

In [6]:
model = service.get_model('ar-AR_BroadbandModel').get_result()
#model = service.get_model('en-US_BroadbandModel').get_result()
print(json.dumps(model, indent=2))

{
  "rate": 16000,
  "name": "ar-AR_BroadbandModel",
  "language": "ar-AR",
  "url": "https://gateway-lon.watsonplatform.net/speech-to-text/api/v1/models/ar-AR_BroadbandModel",
  "supported_features": {
    "custom_language_model": false,
    "speaker_labels": false
  },
  "description": "Modern Standard Arabic broadband model."
}


In [14]:
cwd = cwd +'/firstout.wav'
cwd

'C:\\Users\\MaxTak\\Desktop\\pypro\\ahkh/firstout.wav'

In [19]:

with open(join(dirname('t.txt'), cwd ),
          'rb') as audio_file:
    print(json.dumps(
        service.recognize(
            audio=audio_file,
            content_type='audio/wav',
            model='ar-AR_BroadbandModel',
            timestamps=True,
            word_confidence=False).get_result(),
        indent=2))

{
  "results": [
    {
      "alternatives": [
        {
          "timestamps": [
            [
              "\u0627\u064a\u0646",
              0.11,
              0.32
            ],
            [
              "\u0627\u0644\u062f\u0648\u0644\u0627\u0631\u0627\u062a",
              0.32,
              0.89
            ],
            [
              "\u0633\u0646\u0648\u064a\u0627",
              0.89,
              1.58
            ],
            [
              "\u0644\u0644\u062f\u0641\u0627\u0639",
              1.72,
              2.51
            ],
            [
              "\u0639\u0646",
              2.51,
              2.75
            ],
            [
              "\u0645\u0635\u0627\u0644\u062d\u0647\u0627",
              2.75,
              3.56
            ],
            [
              "\u0645\u0639",
              3.56,
              3.78
            ],
            [
              "\u0627\u0644\u0645\u0624\u062b\u0631\u064a\u0646",
              3.78,
           